# Import libraries

In [ ]:
import numpy as np
import qutip as qtp
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.legend_handler import HandlerTuple
from qutip import *
from functools import cmp_to_key
from tqdm.notebook import tqdm
%matplotlib inline
import ROfunctions as ro
from ROfunctions import *
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import scqubits as scq
from scipy import constants
from scipy import interpolate
import scipy

import h5py
import json
import os
from pathlib import Path
from quantify_core.data.handling import set_datadir, get_tuids_containing, extract_parameter_from_snapshot, load_snapshot, load_quantities_of_interest, load_dataset, to_gridded_dataset, load_processed_dataset

# Set quantify data directory
set_datadir("O:\FPA_RO_Data")

In [ ]:
from matplotlib.colors import PowerNorm, ListedColormap
# Choose colormap
cmap_blue = plt.cm.Blues
cmap_red = plt.cm.Reds
cmap_green = plt.cm.Greens
cmap_purple = plt.cm.Purples
cmap_orange = plt.cm.Oranges

# Get the colormap colors
cmap_blue = cmap_blue(np.arange(plt.cm.Blues.N))
cmap_red = cmap_red(np.arange(plt.cm.Reds.N))
cmap_green = cmap_green(np.arange(plt.cm.Greens.N))
cmap_purple = cmap_purple(np.arange(plt.cm.Purples.N))
cmap_orange = cmap_orange(np.arange(plt.cm.Oranges.N))

# Set alpha
cmap_blue[:,-1] = np.linspace(0, 1, plt.cm.Blues.N)
cmap_red[:,-1] = np.linspace(0, 1, plt.cm.Reds.N)
cmap_green[:,-1] = np.linspace(0, 1, plt.cm.Greens.N)
cmap_purple[:,-1] = np.linspace(0, 1, plt.cm.Purples.N)
cmap_orange[:,-1] = np.linspace(0, 1, plt.cm.Oranges.N)

# Create new colormap
cmap_blue = ListedColormap(cmap_blue)
cmap_red = ListedColormap(cmap_red)
cmap_green = ListedColormap(cmap_green)
cmap_purple = ListedColormap(cmap_purple)
cmap_orange = ListedColormap(cmap_orange)

# Import data
removes nan's resulting from arrays with different lengths in dataframe

In [ ]:
df = pd.read_csv(r'processed_data_fx8.csv')

#Phi_exts
phi_ext_qubit = df['phi_ext_qubit'].to_numpy()
phi_ext_qubit = phi_ext_qubit[~np.isnan(phi_ext_qubit)]
phi_ext_t1 = df['phi_ext_t1'].to_numpy()
phi_ext_t1 = phi_ext_t1[~np.isnan(phi_ext_t1)]

#Corresponds to phi_ext_qubit
qubit_freq = df['qubit_freq'].to_numpy()
qubit_freq = qubit_freq[~np.isnan(qubit_freq)]

#Corresponds to phi_ext_t1
T1 = df['T1'].to_numpy()
T1 = T1[~np.isnan(T1)]

#Fitted energy parameters - E_osc, g, Ej, Ec, El
energy_params = df['energy_params'].to_numpy()
energy_params = energy_params[~np.isnan(energy_params)]

In [ ]:
df1 = pd.read_csv(r'res_fit_results.csv')

#phi_exts
phi_ext_disshift = df1['phi_ext_disshift'].to_numpy()
phi_ext_disshift = phi_ext_disshift[~np.isnan(phi_ext_disshift)]

#resonator frequencies (GHz)
frs_0_all = df1['fr_q0'].to_numpy()
frs_0_all = frs_0_all[~np.isnan(frs_0_all)]
frs_1_all = df1['fr_q1'].to_numpy()
frs_1_all = frs_1_all[~np.isnan(frs_1_all)]
kappa = df1['kappa_avg'].to_numpy()
kappa = kappa[~np.isnan(kappa)]

#chi (MHz)
chi_exp = df1['chi'].to_numpy()
chi_exp = chi_exp[~np.isnan(chi_exp)]

#resonator frequency vs AC flux pulse amplitude
fp_amp = df1['flux_pulse_amp'].to_numpy()
fp_amp = fp_amp[~np.isnan(fp_amp)]

fr_q0_fp = df1['fr_q0_flux_pulse'].to_numpy()
fr_q0_fp = fr_q0_fp[~np.isnan(fr_q0_fp)]

# Theory and numerics

In [ ]:
#states will take the form |qubit> tensor |resonator>
N = 30 #Hilbert space size for qubit
M = 5 #Hilbert space size for resonator

#ladder operators for qubit
c = destroy(N)
cdag = create(N)

#ladder operators for resonator
a = destroy(M)
adag = create(M)

#resonator number operator
rnum = adag * a
resonator_num = tensor(qeye(N), rnum)

#energy parameters
w = energy_params[0] #resonator frequency (GHz)

Ej = energy_params[2] #Josephson energy of qubit (GHz)
Ec = energy_params[3] #capacitive energy of qubit (GHz)
El = energy_params[4] #inductive energy of qubit (GHz)

g = energy_params[1] #coupling strength (GHz)

#Hamiltonian terms
H_lc = w * (adag * a + 1/2) #resonator/LC oscillator term

coupling1 = tensor(c, adag)
coupling2 = tensor(cdag, a)
H_i = g * (coupling1 + coupling2) #interaction term

phi_naught = ((8 * Ec) / El)**(1/4) #oscillator length
n_op = (-1j / (math.sqrt(2) * phi_naught)) * (c - cdag) #charge operator
phi_op = (phi_naught / math.sqrt(2)) * (c + cdag) #flux operator
phi_op_HC = phi_op.dag() #Hermitian conjugate of flux operator

Cterm = 4 * Ec * (n_op)**2
Lterm = (1/2) * El * phi_op**2

#Flux related
mfq = 2 * np.pi #magnetic flux quantum
phi_ext = np.linspace(0, mfq, 1001)

In [ ]:
qubitEdiff_r0 = []
qubitEdiff_r1 = []
resEdiff_q0 = []
resEdiff_q1 = []
chi = [] #MHz

#loop over external flux
for phi in phi_ext:
    Jterm = -Ej * ((1/2) * ((1j * (phi_op - phi)).expm()) + (1/2) * ((-1j * (phi_op_HC - phi)).expm()))
    H_flux = Jterm + Cterm + Lterm
    H_sys = tensor(H_flux, qeye(M)) + tensor(qeye(N), H_lc) + H_i
    states, energies, frequencies, chi_value, truncated_H = truncate_disshift(H_sys, resonator_num)
    
    qubitEdiff_r0.append(frequencies[0].real)
    qubitEdiff_r1.append(frequencies[1].real)
    resEdiff_q0.append(frequencies[2].real)
    resEdiff_q1.append(frequencies[3].real)
    chi.append(chi_value.real * 1000)

In [ ]:
detuning10 = []
detuning20 = []
detuning21 = []
detuning30 = []
detuning31 = []

#loop over external flux
for phi in phi_ext:
    Jterm = -Ej * ((1/2) * ((1j * (phi_op - phi)).expm()) + (1/2) * ((-1j * (phi_op_HC - phi)).expm()))
    H_flux = Jterm + Cterm + Lterm
    H_sys = tensor(H_flux, qeye(M)) + tensor(qeye(N), H_lc) + H_i
    
    states, energies, frequencies, chi_value = truncate_detuning(H_sys, resonator_num)
    d10 = frequencies[0] - w
    d20 = frequencies[1] - w
    d21 = frequencies[2] - w
    d30 = frequencies[3] - w
    d31 = frequencies[4] - w
    detuning10.append(d10.real)
    detuning20.append(d20.real)
    detuning21.append(d21.real)
    detuning30.append(d30.real)
    detuning31.append(d31.real)

## Plotting (full spectrum)

In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
#1D plot of dispersive shift vs external flux
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.4, hspace=0.4)

fig.set_size_inches(3.40457, 2.0)
font = {'size' : 8}
plt.rc('font', **font)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)
ax.tick_params(width=0.5, which='both')
ax.tick_params(length=2.5, which='major')
ax.tick_params(length=1.5, which='minor')

ax.plot(phi_ext / mfq, np.array(chi), linewidth=1.25)

ax.set_xlim(0, 1)
ax.set_ylim(-15,15)

ax.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax.set_ylabel(r"$\chi/2\pi$ (MHz)")

plt.show()

In [ ]:
#1D plot of qubit frequency vs external flux
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.4, hspace=0.4)

fig.set_size_inches(3.40457, 2.0)
font = {'size' : 6}
mpl.rc('font', **font)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)
ax.tick_params(width=0.5, which='both')
ax.tick_params(length=2.5, which='major')
ax.tick_params(length=1.5, which='minor')

ax.plot(phi_ext / mfq, np.array(qubitEdiff_r0), linewidth=1.25)

ax.set_xlim(0, 1)
ax.set_ylim(0,7)

ax.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax.set_ylabel(r"$\omega_{10}/2\pi$ (GHz)")

In [ ]:
#1D plot of resonator frequency vs external flux
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.4, hspace=0.4)

fig.set_size_inches(3.40457, 2.0)
font = {'size' : 8}
mpl.rc('font', **font)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(0.5)
ax.tick_params(width=0.5, which='both')
ax.tick_params(length=2.5, which='major')
ax.tick_params(length=1.5, which='minor')

ax.plot(phi_ext / mfq, np.array(resEdiff_q0), linestyle='-', marker='', color=cmap_red(0.9), label='Qubit 0')
ax.plot(phi_ext / mfq, np.array(resEdiff_q1), linestyle='-', marker='', color=cmap_blue(0.9), label='Qubit 1')
ax.set_xlim(0, 1)
ax.set_ylim(5.17, 5.19)
ax.legend(frameon=True)

ax.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax.set_ylabel(r"$\omega_{r}/2\pi$ (GHz)")

In [ ]:
#1D plot of various qubit level detunings with resonator vs external flux
fig, ax = plt.subplots(ncols=2, nrows=2, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.8, hspace=0.4)

fig.set_size_inches(3.40457, 2.0)
font = {'size' : 6}
mpl.rc('font', **font)

ax1 = ax[0,0]
ax2 = ax[0,1]
ax3 = ax[1,0]
ax4 = ax[1,1]

ax1.plot(phi_ext / mfq, np.array(detuning20), linestyle='', marker='.', markersize=2, color=cmap_green(0.6))
ax1.set_xlim(0.5, 0.8)
ax1.set_ylim(-2, 5)
ax1.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax1.set_ylabel(r"$\Delta_{20}$ (GHz)", labelpad=2)

ax2.plot(phi_ext / mfq, np.array(detuning21), linestyle='', marker='.', markersize=2, color=cmap_blue(0.6))
ax2.set_xlim(0.5, 0.8)
ax2.set_ylim(-5, 0)
ax2.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax2.set_ylabel(r"$\Delta_{21}$ (GHz)", labelpad=2)

ax3.plot(phi_ext / mfq, np.array(detuning30), linestyle='', marker='.', markersize=2, color=cmap_orange(0.6))
ax3.set_xlim(0.5, 0.8)
ax3.set_ylim(0, 6)
ax3.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax3.set_ylabel(r"$\Delta_{30}$ (GHz)", labelpad=2)

ax4.plot(phi_ext / mfq, np.array(detuning31), linestyle='', marker='.', markersize=2, color=cmap_purple(0.6))
ax4.set_xlim(0.5, 0.8)
ax4.set_ylim(-2, 2)
ax4.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax4.set_ylabel(r"$\Delta_{31}$ (GHz)", labelpad=2)

## Plotting (0.5 to 0.8 $\Phi_{ext}$/$\Phi_{0}$)

In [ ]:
phi_ext_clipped = phi_ext[500:801]
chi_clipped = chi[500:801]
fres_q0_clipped = resEdiff_q0[500:801]
fres_q1_clipped = resEdiff_q1[500:801]

In [ ]:
#split chi into subsections
right_idx_chi = []

for i in range(1, len(chi_clipped)):
    x = chi_clipped[i]
    y = chi_clipped[i-1]
    diff = np.abs(x-y)
    product = x * y
    if diff > 5 and (product < 0):
        right_idx_chi.append(i)
        
chi_subsets = np.split(chi_clipped, right_idx_chi)
phi_ext_subsets = np.split(phi_ext_clipped, right_idx_chi)

chi_sub0 = chi_subsets[0]
phi_ext_sub0 = phi_ext_subsets[0]
chi_sub1 = chi_subsets[1]
phi_ext_sub1 = phi_ext_subsets[1]
chi_sub2 = chi_subsets[2]
phi_ext_sub2 = phi_ext_subsets[2]

In [ ]:
#split resonator frequency for qubit in 0 into subsections
right_idx_fresq0 = []

for i in range(1, len(fres_q0_clipped)):
    x = fres_q0_clipped[i]
    y = fres_q0_clipped[i-1]
    diff = np.abs(x-y)
    product = x * y
    if diff > 0.03 and y > x:
        right_idx_fresq0.append(i)
    else:
        continue
        
fres_q0_subsets = np.split(fres_q0_clipped, right_idx_fresq0)
phi_ext_frq0_subsets = np.split(phi_ext_clipped, right_idx_fresq0)

fres_q0_sub0 = fres_q0_subsets[0]
phi_ext_frq0_sub0 = phi_ext_frq0_subsets[0]
fres_q0_sub1 = fres_q0_subsets[1]
phi_ext_frq0_sub1 = phi_ext_frq0_subsets[1]

In [ ]:
#split resonator frequency for qubit in 1 into subsections
right_idx_fresq1 = []

for i in range(1, len(fres_q1_clipped)):
    x = fres_q1_clipped[i]
    y = fres_q1_clipped[i-1]
    diff = np.abs(x-y)
    product = x * y
    if diff > 0.05 and y < x:
        right_idx_fresq1.append(i)
    else:
        continue

fres_q1_subsets = np.split(fres_q1_clipped, right_idx_fresq1)
phi_ext_frq1_subsets = np.split(phi_ext_clipped, right_idx_fresq1)

fres_q1_sub0 = fres_q1_subsets[0]
phi_ext_frq1_sub0 = phi_ext_frq1_subsets[0]
fres_q1_sub1 = fres_q1_subsets[1]
phi_ext_frq1_sub1 = phi_ext_frq1_subsets[1]

In [ ]:
# from left to right
# x coordinates for v lines in chi plot
phi_ext_1 = (phi_ext_clipped[right_idx_chi[0]] + phi_ext_clipped[right_idx_chi[0]-1]) / (2*mfq)
phi_ext_2 = (phi_ext_clipped[right_idx_chi[1]] + phi_ext_clipped[right_idx_chi[1]-1]) / (2*mfq)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, dpi=250, sharex=True)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.6, hspace=0.25)
fig.set_size_inches(3.40457*1.25, 3.0/2)
font = {'size' : 6}
mpl.rc('font', **font)

chi_color=cmap_purple(0.8)
color_31 =cmap_blue(0.7)
color_20 =cmap_red(0.7)

ax1.vlines(phi_ext_1, -10, 10, color=color_31, linewidth=0.9, ls='dashed', label=r"$\Delta_{31} \rightarrow 0$")
ax1.vlines(phi_ext_2, -10, 10, color=color_20, linewidth=0.9, ls='dashed', label=r"$\Delta_{20} \rightarrow 0$")
ax1.plot(phi_ext_sub0/mfq, chi_sub0, color=chi_color, linewidth=1.25, label=r"$\chi/2\pi$")
ax1.plot(phi_ext_sub1/mfq, chi_sub1, color=chi_color, linewidth=1.25)
ax1.plot(phi_ext_sub2/mfq, chi_sub2, color=chi_color, linewidth=1.25)
ax1.set_xlim(0.5, 0.8)
ax1.set_ylim(-10,10)
ax1.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax1.set_ylabel(r"$\chi/2\pi$ (MHz)", labelpad=-2)
ax1.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.4, 1))

res_q0_color=cmap_red(0.9)
res_q1_color=cmap_blue(0.9)

ax2.vlines(phi_ext_1, 5.1, 5.3, color=color_31, linewidth=0.9, ls='dashed', label=r"$\Delta_{31} \rightarrow 0$")
ax2.vlines(phi_ext_2, 5.1, 5.3, color=color_20, linewidth=0.9, ls='dashed', label=r"$\Delta_{20} \rightarrow 0$")

ax2.plot(phi_ext_frq0_sub0/mfq, fres_q0_sub0, color=res_q0_color, linewidth=1.25, label=r"q=$\left|{0}\right\rangle$")
ax2.plot(phi_ext_frq0_sub1/mfq, fres_q0_sub1, color=res_q0_color, linewidth=1.25)

ax2.plot(phi_ext_frq1_sub0/mfq, fres_q1_sub0, color=res_q1_color, linewidth=1.25, label=r"q=$\left|{1}\right\rangle$")
ax2.plot(phi_ext_frq1_sub1/mfq, fres_q1_sub1, color=res_q1_color, linewidth=1.25)

ax2.set_xlim(0.5, 0.8)
ax2.set_ylim(5.16,5.19)
ax2.set_xlabel(r"$\Phi_{ext}/\Phi_0$")
ax2.set_ylabel(r"$\omega_{r}/2\pi$ (GHz)")
ax2.legend(frameon=False,loc='upper center', bbox_to_anchor=(0.35, 1.03))

plt.show()

# Compare numerics with experiment

## Fit T1 data with dielectric loss model

In [ ]:
hbar = constants.hbar #Js
kb = constants.k #J/K
Phi0 = constants.physical_constants['mag. flux quantum'][0]
phi0 = Phi0/(2*np.pi*hbar)

phis = np.linspace(0.5, 1, 501)
phi01 = []
for i in phis:
    qubit = scq.Fluxonium(EJ = Ej, EC = Ec, EL = El, flux = i, cutoff = 100)
    phi_elem = np.abs(qubit.matrixelement_table('phi_operator')[0, 1])
    phi01.append(phi_elem)

phi01_func_phiext = interpolate.interp1d(phis, phi01, fill_value="extrapolate")
fqubit_func_phiext = interpolate.interp1d(phis, np.array(qubitEdiff_r0[500:]), fill_value="extrapolate")

In [ ]:
#loss_tan_ref unitless
#pwr unitless

def T1_model(phi, loss_tan_ref, pwr):
    Teff = 0.02 #K
    phi_matelem = phi01_func_phiext(phi)
    freq01_ghz = fqubit_func_phiext(phi)
    omega01 = 2*np.pi*freq01_ghz*10**9 #converts to 2pi*Hz
    EC = hbar*energy_params[3]*10**9
    EL = hbar*energy_params[4]*10**9
    ref_freq = 2*np.pi*1e9
    
    coth_arg = (hbar*omega01)/(2*kb*Teff)
    gamma_diel = ((hbar*omega01**2)/(4*EC))*(phi_matelem**2)*loss_tan_ref*(omega01/ref_freq)**(pwr)*(np.cosh(coth_arg)/np.sinh(coth_arg))
    
    T1 = 1/(gamma_diel)
    return T1

In [ ]:
t1_sec = np.array(T1)*10**(-6)
parameter_bounds = ([1e-9, 0.01], [1e-4, 5])

popt_t1, pcov_t1 = scipy.optimize.curve_fit(
    T1_model,
    phi_ext_t1,
    t1_sec,
    p0=(2e-6, 0.2),
    sigma=[*(1e-10,)*len(t1_sec)],
    bounds=parameter_bounds
)
print("Success")
print(popt_t1)

fitted_t1 = T1_model(phi_ext_t1, *popt_t1)*10**6

### Visualize qubit frequency and T1

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, dpi=250, sharex=True)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.5, hspace=0.25)
fig.set_size_inches(3.40457, 3.0/2)
font = {'size' : 8}
mpl.rc('font', **font)

ax1.plot(phi_ext_qubit, qubit_freq, label = "Experimental", color=cmap_blue(0.5), linestyle='', marker='.', markersize=2)
ax1.plot(np.linspace(0, 1, 1001), np.array(qubitEdiff_r0), linewidth=1, label = "Numerical", color=cmap_blue(0.9), linestyle='dashed')
ax1.set_ylabel(r"$\omega_{q}$ (GHz)")
ax1.set_xlabel(r"$\Phi_{ext}$/$\Phi_{0}$")
ax1.set_xlim(0.5, 1)
ax1.set_ylim(0, 5)
ax1.legend(fontsize=6, frameon=False)

ax2.plot(phi_ext_t1, T1, color=cmap_red(0.7), linestyle='', marker='.', markersize=4, label='Experimental')
ax2.plot(phi_ext_t1, fitted_t1, color=cmap_red(0.9), linestyle='--', label='Numerical')
ax2.set_ylabel(r"$T_{1}$ ($\mu s$)")
ax2.set_xlabel(r"$\Phi_{ext}$/$\Phi_{0}$")
ax2.tick_params(axis='y', pad=1.5)

plt.show()

## Dispersive shift and resonator frequency

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, dpi=250, sharex=True)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.16, right=0.9, wspace=0.5, hspace=0.25)
fig.set_size_inches(3.5, 1.25)
font = {'size' : 6}
mpl.rc('font', **font)

ticklabel = 8
axislabel = 8
legendtext = 6

chi_color_pred=cmap_purple(0.7)
chi_color_exp=cmap_purple(0.9)
fresq0_color_pred=cmap_red(0.7)
fresq0_color_exp=cmap_red(0.9)
fresq1_color_pred=cmap_blue(0.7)
fresq1_color_exp=cmap_blue(0.9)

ax1.plot(phi_ext_sub0/mfq, chi_sub0, color=chi_color_pred, linewidth=1.5, linestyle='-', label=r"Numerics")
ax1.plot(phi_ext_sub1/mfq, chi_sub1, color=chi_color_pred, linestyle='-', linewidth=1.5)
ax1.plot(phi_ext_sub2/mfq, chi_sub2, color=chi_color_pred, linestyle='-', linewidth=1.5)
ax1.plot(phi_ext_disshift, chi_exp, color=chi_color_exp, linestyle='', marker='.', markersize=4, label=r"Data")
ax1.set_xticks([0.5, 0.6, 0.7, 0.8])
ax1.set_yticks([-10, -5, 0, 5, 10])
ax1.tick_params(axis='y', pad=3, labelsize=ticklabel)
ax1.tick_params(axis='x', pad=3, labelsize=ticklabel)
ax1.set_xlim(0.5, 0.8)
ax1.set_ylim(-10, 10)
ax1.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=axislabel)
ax1.set_ylabel(r"$\chi/2\pi$ (MHz)", fontsize=axislabel, labelpad=0)
ax1.legend(handletextpad=0.5, frameon=False, handlelength = 1.0, fontsize = legendtext, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(-0.05, -0.05))

ax2.vlines(phi_ext_1, 5.1, 5.3, color=color_31, linewidth=0.9, ls='dotted', label=r"$\Delta_{31} \rightarrow 0$")
ax2.vlines(phi_ext_2, 5.1, 5.3, color=color_20, linewidth=0.9, ls='dotted', label=r"$\Delta_{20} \rightarrow 0$")
ax2.plot(phi_ext_frq0_sub0/mfq, fres_q0_sub0, color=fresq0_color_pred, linewidth=1.5, linestyle='-')
ax2.plot(phi_ext_frq0_sub1/mfq, fres_q0_sub1, color=fresq0_color_pred, linewidth=1.5, linestyle='-')
ax2.plot(phi_ext_frq1_sub0/mfq, fres_q1_sub0, color=fresq1_color_pred, linewidth=1.5, linestyle='-')
ax2.plot(phi_ext_frq1_sub1/mfq, fres_q1_sub1, color=fresq1_color_pred, linewidth=1.5, linestyle='-')
ax2.plot(phi_ext_disshift, frs_0_all, color=fresq0_color_exp, linestyle='', marker='.', markersize=4, label=r"q=$\left|{0}\right\rangle$")
ax2.plot(phi_ext_disshift, frs_1_all, color=fresq1_color_exp, linestyle='', marker='.', markersize=4, label=r"q=$\left|{1}\right\rangle$")
ax2.set_xlim(0.5, 0.8)
ax2.set_ylim(5.16, 5.19)
ax2.set_xticks([0.5, 0.6, 0.7, 0.8])
ax2.set_yticks([5.16, 5.17, 5.18, 5.19])
ax2.tick_params(axis='y', pad=3, labelsize=ticklabel)
ax2.tick_params(axis='x', pad=3, labelsize=ticklabel)
ax2.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=axislabel)
ax2.set_ylabel(r"$\omega_{r}/2\pi$ (GHz)", fontsize=axislabel, labelpad=2)
ax2.legend(handletextpad=0.5, frameon=False, handlelength = 1.0, fontsize = legendtext, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(0.05, -0.05))

# Combine spectroscopy and dispersive shift plots

In [ ]:
fig = plt.figure(figsize=(3.40457, 3.0))

gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1.5], wspace=0.45, hspace=0.38, bottom = 0.1, top=0.9, left=0.12, right=0.9,)

label_fs = 7
tick_fs = 7
legend_fs = 6

fqubit_color_pred = cmap_green(0.9)
fqubit_color_exp = cmap_green(0.5)
chi_color_pred=cmap_purple(0.7)
chi_color_exp=cmap_purple(0.9)
fresq0_color_pred=cmap_red(0.9)
fresq0_color_exp=cmap_red(0.6)
fresq1_color_pred=cmap_blue(0.9)
fresq1_color_exp=cmap_blue(0.6)

ax1 = plt.subplot(gs[0, 0])
ax1.plot(phi_ext_qubit, qubit_freq, label = "Data", color=fqubit_color_exp, linestyle='', marker='.', markersize=6) #, markersize=6)
ax1.plot(np.linspace(0, 1, 1001), np.array(qubitEdiff_r0), linewidth=2, label = "Model", color=fqubit_color_pred, linestyle='dashed')
ax1.set_ylabel(r"$\omega_{q}/2\pi$ (GHz)", fontsize=label_fs)
ax1.set_xlabel(r"$\Phi_{ext}$/$\Phi_{0}$", fontsize=label_fs, labelpad=1)
ax1.set_yticks([0, 1, 2, 3, 4, 5], labels=['0', '1', '2', '3', '4', '5'], fontsize=tick_fs)
ax1.set_xticks([0.5, 0.6, 0.7, 0.8], labels=['0.5', '0.6', '0.7', '0.8'], fontsize=tick_fs)
ax1.tick_params(axis='y', pad=2)
ax1.set_xlim(0.5, 0.8)
ax1.set_ylim(0, 5)
ax1.legend(loc='lower left', bbox_to_anchor=(0.46, -0.03), handletextpad=0.2, frameon=False, markerscale=1, fontsize=legend_fs, handlelength=1.1)
ax1.annotate("(a)", xy=(0.01, 1.05), xycoords="axes fraction", fontsize=label_fs)


ax2 = plt.subplot(gs[0, 1])
a = ax2.vlines(phi_ext_1, 5.1, 5.3, color=color_31, linewidth=1.5, ls='dotted', label=r"$\Delta_{31} \rightarrow 0$")
b = ax2.vlines(phi_ext_2, 5.1, 5.3, color=color_20, linewidth=1.5, ls='dotted', label=r"$\Delta_{20} \rightarrow 0$")
g, = ax2.plot(phi_ext_disshift, frs_0_all, color=fresq0_color_exp, linestyle='', marker='.', label=r"q=$\left|{0}\right\rangle$", markersize=6)
h, = ax2.plot(phi_ext_disshift, frs_1_all, color=fresq1_color_exp, linestyle='', marker='.', label=r"q=$\left|{1}\right\rangle$", markersize=6)
c, = ax2.plot(phi_ext_frq0_sub0/mfq, fres_q0_sub0, color=fresq0_color_pred, linewidth=2, linestyle='dashed')
d, = ax2.plot(phi_ext_frq0_sub1/mfq, fres_q0_sub1, color=fresq0_color_pred, linewidth=2, linestyle='dashed')
e, = ax2.plot(phi_ext_frq1_sub0/mfq, fres_q1_sub0, color=fresq1_color_pred, linewidth=2, linestyle='dashed')
f, = ax2.plot(phi_ext_frq1_sub1/mfq, fres_q1_sub1, color=fresq1_color_pred, linewidth=2, linestyle='dashed')

ax2.set_xlim(0.5, 0.8)
ax2.set_ylim(5.16, 5.19)
ax2.set_xticks([0.5, 0.6, 0.7, 0.8], labels=['0.5', '0.6', '0.7', '0.8'], fontsize=tick_fs)
ax2.set_yticks([5.16, 5.17, 5.18, 5.19], labels=['5.16', '5.17', '5.18', '5.19'], fontsize=tick_fs)
ax2.tick_params(axis='y', pad=1.5)
ax2.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=label_fs, labelpad=1)
ax2.set_ylabel(r"$\omega_{r}/2\pi$ (GHz)", fontsize=label_fs, labelpad=1)
l = ax2.legend([(c, g), (e, h), b, a], [r"q=$\left|{0}\right\rangle$", r"q=$\left|{1}\right\rangle$", r"$\Delta_{20} \rightarrow 0$", r"$\Delta_{31} \rightarrow 0$"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=0.2, frameon=False, handlelength = 1.4, labelspacing=0.2, fontsize=legend_fs,
               loc='lower left', bbox_to_anchor=(0.08, -0.08))
ax2.annotate("(b)", xy=(0.01, 1.05), xycoords="axes fraction", fontsize=label_fs)

ax3 = plt.subplot(gs[1, :])
b = ax3.plot(phi_ext_sub0/mfq, chi_sub0, color=chi_color_pred, linewidth=2, linestyle='-', label=r"Model")
ax3.plot(phi_ext_sub1/mfq, chi_sub1, color=chi_color_pred, linestyle='-', linewidth=2)
ax3.plot(phi_ext_sub2/mfq, chi_sub2, color=chi_color_pred, linestyle='-', linewidth=2)
a = ax3.plot(phi_ext_disshift, chi_exp, color=chi_color_exp, linestyle='', marker='.', markersize=6, label=r"Data")
ax3.set_xticks([0.5, 0.6, 0.7, 0.8])
ax3.set_yticks([-10, -5, 0, 5, 10])
ax3.tick_params(axis='y', pad=1.5, labelsize=tick_fs)
ax3.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax3.set_xlim(0.5, 0.8)
ax3.set_ylim(-10, 10)
ax3.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=label_fs, labelpad=1)
ax3.set_ylabel(r"$\chi/2\pi$ (MHz)", fontsize=label_fs, labelpad=-2)
handles, labels = ax3.get_legend_handles_labels()

#specify order of items in legend
order = [1, 0]

#add legend to plot
ax3.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', bbox_to_anchor=(-0.01, 0.3), handletextpad=0.4, frameon=False, markerscale=1.2, fontsize=legend_fs, handlelength=1.1)

ax3.annotate("(c)", xy=(0.01, 1.05), xycoords="axes fraction", fontsize=label_fs)
plt.tight_layout()